# Model for an online book selling website

This model has been trained only on two books for demonstration but can be trained on many more, if we want.
These two books are Rich Dad and Poor Dad, The Power of Your Subconscious Mind.

In [ ]:
# Convolutional Neural Network

# Importing the libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

# Part 1 - Data Preprocessing

# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Train_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

# Part 2 - Building the CNN

# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

# Part 4 - Making a single prediction

import numpy as np
from keras.preprocessing import image
test_image = image.load_img('new_test_image.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'RichDadPoorDad'
else:
    prediction = 'PowerOfYourSubconsciousMind'
print(prediction)

Found 8 images belonging to 2 classes.
Found 2 images belonging to 2 classes.
Epoch 1/25
1/1 [==============================] - 0s 306ms/step - loss: 0.6713 - accuracy: 0.5000 - val_loss: 0.7465 - val_accuracy: 0.5000
Epoch 2/25
1/1 [==============================] - 0s 77ms/step - loss: 0.7002 - accuracy: 0.5000 - val_loss: 0.6442 - val_accuracy: 1.0000
Epoch 3/25
1/1 [==============================] - 0s 79ms/step - loss: 0.5720 - accuracy: 0.8750 - val_loss: 0.6483 - val_accuracy: 0.5000
Epoch 4/25
1/1 [==============================] - 0s 79ms/step - loss: 0.5265 - accuracy: 0.8750 - val_loss: 0.6090 - val_accuracy: 0.5000
Epoch 5/25
1/1 [==============================] - 0s 78ms/step - loss: 0.4297 - accuracy: 0.8750 - val_loss: 0.6249 - val_accuracy: 0.5000
Epoch 6/25
1/1 [==============================] - 0s 79ms/step - loss: 0.3868 - accuracy: 0.7500 - val_loss: 0.6018 - val_accuracy: 0.5000
Epoch 7/25
1/1 [==============================] - 0s 80ms/step - loss: 0.3211 - accurac

In [ ]:
if prediction=='RichDadPoorDad':
    refFilename = 'base_image_rich.jpg'
else:
    refFilename = 'base_image_power.jpg'

Image Alignment using ORB and RANSAC,
sift or surf can also be used in place of ORB in a similar manner.

The image containing the matches and final image after alignment are created below and are present in the Folder containing this file as well

In [ ]:
import cv2
from __future__ import print_function
import numpy as np

MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

def alignImages(im1, im2):

  # Convert images to grayscale
  im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(MAX_FEATURES)
  keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = matcher.match(descriptors1, descriptors2, None)

  # Sort matches by score
  matches = sorted(matches,key=lambda x: x.distance, reverse=False)

  # Remove not so good matches
  numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
  matches = matches[:numGoodMatches]

  # Draw top matches
  imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  cv2.imwrite("matches.jpg", imMatches)

  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)

  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

  # Use homography
  height, width, channels = im2.shape
  im1Reg = cv2.warpPerspective(im1, h, (width, height))

  return im1Reg, h

if __name__ == '__main__':

  # Read reference image
  print("Reading reference image : ", refFilename)
  imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

  # Read image to be aligned

  print("Aligning images ...")
  # Registered image will be resotred in imReg.
  # The estimated homography will be stored in h.
  imReg, h = alignImages(cv2.imread('new_test_image.jpg', cv2.IMREAD_COLOR), imReference)

  # Write aligned image to disk.
  outFilename = "aligned.jpg"
  print("Saving aligned image : ", outFilename);
  cv2.imwrite(outFilename, imReg)

  # Print estimated homography
  print("Estimated homography : \n",  h)

Reading reference image :  base_image_rich.jpg
Aligning images ...
Saving aligned image :  aligned.jpg
Estimated homography : 
 [[ 7.37782366e-01 -7.06815630e-01  1.59947729e+02]
 [ 4.49344536e-01  7.49280232e-01  4.46194136e+01]
 [-3.55399201e-04 -3.72163188e-04  1.00000000e+00]]


**Conclusion**:- Image Alignment does not function that efficiently as that crops and resize the image based upon the features, and that depends on our test image based on which may be we can get very good result and may be we can get not so satisfying result.